In [41]:

from pandas_datareader import data as pdr
import yfinance as yfm
from tqdm import tqdm
yf.pdr_override()

ticker_df_list =list(data['Ticker'].unique())
price_data={}
for ticker in ticker_df_list:
    df = yf.download(ticker, interval='1d', period='3y')
    price_data[ticker] = df
    df.to_csv(f'stockPrice/{ticker}.csv')
    break

[*********************100%%**********************]  1 of 1 completed


In [1]:

import os
import pandas as pd
from tqdm import tqdm
# Set the path to the directory containing the CSV files
PATH = '/home/ssu36/tiger/Stock_Prediction/data/stockPrice'

# Get the list of CSV files in the directory
csv_files = [file for file in os.listdir(PATH) if file.endswith('.csv')]

# Initialize an empty dictionary to store the dataframes
dfs = {}

# Iterate over each CSV file
for file in tqdm(csv_files):
  # Read the CSV file into a dataframe
  df = pd.read_csv(os.path.join(PATH, file))
  ticker_name =file.split('.')[0]
  df["Ticker"] = ticker_name
  # Add the dataframe to the dictionary with the file name as the key
  if len(df) == 755:
    dfs[ticker_name] = df
data = pd.concat(dfs.values(), ignore_index=True)
# Print the dictionary of dataframes


100%|██████████| 2743/2743 [00:04<00:00, 561.37it/s]


In [2]:
import datasets
data= datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y',split='train')
data = data.to_pandas()


def min_max(sequences):
  results = sequences.copy()
  v_min =results.min()
  v_max =results.max()
  new_min =0
  new_max =1
  min_max=[]
  for index,sequence in enumerate(results):
    v_min =sequence.min()
    v_max =sequence.max()
    v_p = (sequence - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    min_max.append([v_min,v_max])
    results[index] = v_p
  return results, min_max

def inverse_min_max(sequences,min_max):
  original = []
  for index,sequence in enumerate(sequences):
    v_min =min_max[index][0]
    v_max =min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
  return np.array(original)

In [ ]:

def preprocess_window(df,feature:list,window_size:int,predict_size:int,sliding_size:int):
  ticker_number =len(df['Ticker'].unique())
  total_day = len(df['Date'].unique())
  window_number =int(total_day - (window_size+predict_size)/sliding_size+1)
  feature_size = len(feature)

  sequences = list()
  for group in df.groupby('Ticker'):
    sequences.append(group[1][feature])
  sequences=np.array(sequences)

  ## min_max trading
  price , trade,sentiments =np.split(sequences,[4,5],axis=2)
  trade , min_max_trading =min_max(trade)
  price , min_max_list =min_max(price)
  combine =np.concatenate([price,trade,sentiments],axis =2)

  ## min_max another
  result_list= []
  for i in range(0,window_number):
    a, b, c= np.split(combine,[i,i+window_size+predict_size],axis=1)
    result_list.append(b)
  result_array = np.array(result_list)
  train ,vaild = np.split(result_array,[-1],axis=0)
  # train_x,train_y=np.split(train.reshape(total_day%(window_size+predict_size)*ticker_number,window_size+predict_size,feature_size),[window_size],axis=1)
  train_x,train_y=np.split(train.reshape((window_number-1)*ticker_number,window_size+predict_size,feature_size),[window_size],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*ticker_number,window_size+predict_size,feature_size),[window_size],axis=1)
  print(train_x.shape,train_y.shape,vaild_x.shape,vaild_y.shape,len(min_max_list))
  return train_x,train_y,vaild_x,vaild_y,min_max_list


In [119]:
import torch
import numpy as np
feature = [ 'Open', 'High','Low','Close', 'Volume']

sequences = list()
for group in data.groupby('Ticker'):
    sequences.append(group[1][feature])
sequences=np.array(sequences)

In [19]:
price , trade,sentiments =np.split(sequences,[4,5],axis=2)
trade , min_max_trading =min_max(trade)
price , min_max_list =min_max(price)
combine =np.concatenate([price,trade,sentiments],axis =2)

In [ ]:
def inverse_min_max(sequences,min_max):
  original = []
  for index,sequence in enumerate(sequences):
    v_min =min_max[index][0]
    v_max =min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
  return np.array(original)

In [67]:
def inverse_min_max_gpu(sequences, min_max):
    sequences_tensor = torch.tensor(sequences, dtype=torch.float32, device='cuda')
    min_max_tensor = torch.tensor(min_max, dtype=torch.float32, device='cuda')
    original = []
    for index,sequence in enumerate(sequences_tensor):
        v_min =min_max_tensor[index][0]
        v_max =min_max_tensor[index][1]
        sequence_restored = (sequence) * (v_max - v_min) + v_min
        original.append(sequence_restored)
    result =torch.stack(original)
    return result

In [26]:
sequences.shape

(2136, 755, 5)

In [112]:

def make_split(sequences,WINDOW_NUMBER,PREDICT_SIZE,WINDOW_SIZE,FEATURE_SIZE,TICKER_NUMBER):
  original_list =[]
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(sequences,[i,i+WINDOW_SIZE+PREDICT_SIZE],axis=1)
    original_list.append(b)
  print(len(original_list))
  result_array = np.array(original_list)
  train ,vaild = np.split(result_array,[-1],axis=0)
  train_x,train_y=np.split(train.reshape((WINDOW_NUMBER-1)*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  return train_x,train_y,vaild_x,vaild_y

In [120]:
train_x,train_y,vaild_x,vaild_y = make_split(sequences,window_number,predict_size,window_size,feature_size,ticker_number)

651


In [121]:
train_x

array([[[1.17999995e+00, 1.19000006e+00, 1.15999997e+00, 1.16999996e+00,
         3.49000000e+04],
        [1.17999995e+00, 1.25000000e+00, 1.17999995e+00, 1.21000004e+00,
         1.90400000e+05],
        [1.25999999e+00, 1.28999996e+00, 1.22000003e+00, 1.26999998e+00,
         1.99200000e+05],
        ...,
        [4.19999981e+00, 4.75000000e+00, 4.19999981e+00, 4.36999989e+00,
         2.15240000e+06],
        [4.40999985e+00, 4.50000000e+00, 4.17000008e+00, 4.19000006e+00,
         3.82300000e+05],
        [4.15000010e+00, 4.28999996e+00, 4.07000017e+00, 4.19999981e+00,
         1.32100000e+05]],

       [[2.37000008e+01, 2.51849995e+01, 2.32500000e+01, 2.44500008e+01,
         4.70070000e+04],
        [2.46000004e+01, 2.47500000e+01, 2.25000000e+01, 2.29500008e+01,
         5.03600000e+04],
        [2.32500000e+01, 2.41499996e+01, 2.32199993e+01, 2.33999996e+01,
         1.52730000e+04],
        ...,
        [1.97999992e+01, 2.06250000e+01, 1.90200005e+01, 1.93500004e+01,
        

In [ ]:
train_x,train_y,vaild_x,vaild_y = make_split(combine,window_number,predict_size,window_size,feature_size,ticker_number)

In [1]:
from data_processing import Window_maker
import datasets
import torch
torch.tensor(original[0]).shape
data= datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y',split='train')
data = data.to_pandas()
window_size= 90
predict_size= 15
sliding_size=1
window_maked = Window_maker(window_size,sliding_size,predict_size,data)


/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
original, scaled ,min_max_list= window_maked.preprocess_window()

전처리 완료..


In [5]:
result = Window_maker.inverse_min_max(torch.tensor(original[0]),torch.tensor(min_max_list))

IndexError: index 2136 is out of bounds for dimension 0 with size 2136

In [11]:
orginal5 = torch.reshape(torch.tensor(original[0]),(2136,650,90,5))

In [25]:
original[1].shape

(1388400, 15, 5)

In [10]:
torch.tensor(original[0]).shape

torch.Size([1388400, 90, 5])

In [4]:
import torch
torch.tensor(original[0]).shape

torch.Size([1388400, 90, 5])

In [ ]:


SUFFLE = False
WINDOW_SIZE  = 90
PREDICT_SIZE = 15
SLIDING_SIZE =1
FEATURE_SIZE = 5
BATCH_SIZE =64
FULLLY_SIZE =128
FEATURE = [ 'Open', 'High','Low','Close', 'Volume']
TARGET = 'Close'
TICKER_NUMBER =2136
TOTAL_DAY = 755
WINDOW_NUMBER =int(TOTAL_DAY - (WINDOW_SIZE+PREDICT_SIZE)/SLIDING_SIZE+1)



data= datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y')
data = data['train'].to_pandas()


def min_max(sequences):
  results = sequences.copy()
  v_min =results.min()
  v_max =results.max()
  new_min =0
  new_max =1
  min_max=[]
  for index,sequence in enumerate(results):
    v_min =sequence.min()
    v_max =sequence.max()
    v_p = (sequence - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    min_max.append([v_min,v_max])
    results[index] = v_p
  return results, min_max

def inverse_min_max(sequences,min_max):
  original = []
  for index,sequence in enumerate(sequences):
    v_min =min_max[index][0]
    v_max =min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
  return np.array(original)

def preprocess_lstm(df):
  sequences = list()
  for group in df.groupby('Ticker'):
    sequences.append(group[1][FEATURE])
  sequences=np.array(sequences)

  ## min_max trading
  price , trade,sentiments =np.split(sequences,[4,5],axis=2)
  trade , min_max_trading =min_max(trade)
  price , min_max_list =min_max(price)
  combine =np.concatenate([price,trade,sentiments],axis =2)

  ## min_max another
  result_list= []
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(combine,[i,i+WINDOW_SIZE+PREDICT_SIZE],axis=1)
    result_list.append(b)
  result_array = np.array(result_list)
  print(166//(WINDOW_SIZE+PREDICT_SIZE))
  # train ,vaild = np.split(result_array,[TOTAL_DAY%(WINDOW_SIZE+PREDICT_SIZE)],axis=0)
  train ,vaild = np.split(result_array,[-1],axis=0)

  print(train.shape ,vaild.shape)
  # train_x,train_y=np.split(train.reshape(TOTAL_DAY%(WINDOW_SIZE+PREDICT_SIZE)*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  train_x,train_y=np.split(train.reshape((WINDOW_NUMBER-1)*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  return train_x,train_y,vaild_x,vaild_y,min_max_list



class BaseDataset(Dataset):
    def __init__(self, x_data, y_data, target):
        self.x_data = torch.tensor(x_data,dtype =torch.float32)
        self.y_data = torch.tensor(y_data[:,FEATURE.index(target),:],dtype = torch.float32)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.x_data.shape[0]
train_x,train_y,vaild_x,vaild_y,min_max_list = preprocess_lstm(data)

def swap_axes(arr):
    return np.swapaxes(arr, 1, 2)
train_x = swap_axes(train_x)
train_y = swap_axes(train_y)
vaild_x = swap_axes(vaild_x)
vaild_y = swap_axes(vaild_y)
train_data = BaseDataset(train_x,train_y,TARGET)
test_data =  BaseDataset(vaild_x,vaild_y,TARGET)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False,drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False,drop_last=True)
train_x.shape,train_y.shape,vaild_x.shape,vaild_y.shape,len(min_max_list)





NameError: name 'datasets' is not defined

In [4]:
len(min_max_list)

2136

In [3]:
original[0]

array([[[1.17999995e+00, 1.19000006e+00, 1.15999997e+00, 1.16999996e+00,
         3.49000000e+04],
        [1.17999995e+00, 1.25000000e+00, 1.17999995e+00, 1.21000004e+00,
         1.90400000e+05],
        [1.25999999e+00, 1.28999996e+00, 1.22000003e+00, 1.26999998e+00,
         1.99200000e+05],
        ...,
        [4.19999981e+00, 4.75000000e+00, 4.19999981e+00, 4.36999989e+00,
         2.15240000e+06],
        [4.40999985e+00, 4.50000000e+00, 4.17000008e+00, 4.19000006e+00,
         3.82300000e+05],
        [4.15000010e+00, 4.28999996e+00, 4.07000017e+00, 4.19999981e+00,
         1.32100000e+05]],

       [[2.37000008e+01, 2.51849995e+01, 2.32500000e+01, 2.44500008e+01,
         4.70070000e+04],
        [2.46000004e+01, 2.47500000e+01, 2.25000000e+01, 2.29500008e+01,
         5.03600000e+04],
        [2.32500000e+01, 2.41499996e+01, 2.32199993e+01, 2.33999996e+01,
         1.52730000e+04],
        ...,
        [1.97999992e+01, 2.06250000e+01, 1.90200005e+01, 1.93500004e+01,
        

In [4]:
scaled[0]

array([[[2.00527671e-02, 2.05804773e-02, 1.89973591e-02, 1.95250631e-02,
         1.08915286e-04],
        [2.00527671e-02, 2.37467012e-02, 2.00527671e-02, 2.16358853e-02,
         6.01250373e-04],
        [2.42744052e-02, 2.58575172e-02, 2.21635893e-02, 2.48021092e-02,
         6.29112423e-04],
        ...,
        [1.79419514e-01, 2.08443271e-01, 1.79419514e-01, 1.88390495e-01,
         6.81322105e-03],
        [1.90501311e-01, 1.95250659e-01, 1.77836415e-01, 1.78891823e-01,
         1.20883303e-03],
        [1.76781007e-01, 1.84168863e-01, 1.72559375e-01, 1.79419514e-01,
         4.16664292e-04]],

       [[4.31620231e-01, 4.63959054e-01, 4.21820563e-01, 4.47952984e-01,
         2.07224966e-03],
        [4.51219527e-01, 4.54486069e-01, 4.05487810e-01, 4.15287479e-01,
         2.27700752e-03],
        [4.21820563e-01, 4.41419858e-01, 4.21167238e-01, 4.25087105e-01,
         1.34347535e-04],
        ...,
        [3.46689883e-01, 3.64655928e-01, 3.29703847e-01, 3.36890256e-01,
        

In [117]:
result

tensor([[[ 1.1800,  1.1900,  1.1600,  1.1700,  0.8021],
         [ 1.1800,  1.2500,  1.1800,  1.2100,  0.8114],
         [ 1.2600,  1.2900,  1.2200,  1.2700,  0.8119],
         ...,
         [ 0.9300,  0.9300,  0.8700,  0.9100,  0.8001],
         [ 0.9800,  0.9800,  0.9100,  0.9100,  0.8000],
         [ 0.9500,  0.9700,  0.8700,  0.9300,  0.8001]],

        [[23.7000, 25.1850, 23.2500, 24.4500,  3.9752],
         [24.6000, 24.7500, 22.5000, 22.9500,  3.9846],
         [23.2500, 24.1500, 23.2200, 23.4000,  3.8862],
         ...,
         [ 4.9000,  4.9000,  4.7000,  4.7500,  4.0093],
         [ 4.7500,  4.8400,  4.6800,  4.6800,  3.9774],
         [ 4.7300,  4.8000,  4.6900,  4.7500,  3.9880]],

        [[12.7500, 13.5800, 12.6900, 13.5600, 15.3281],
         [14.3200, 14.9700, 14.0800, 14.8200, 18.0691],
         [14.7700, 15.0800, 14.3700, 14.9400, 15.3423],
         ...,
         [12.2300, 12.4500, 12.1700, 12.4000, 11.5472],
         [12.3000, 12.3300, 12.0500, 12.1300, 11.5777],
  

In [114]:
train_x

array([[[2.00527671e-02, 2.05804773e-02, 1.89973591e-02, 1.95250631e-02,
         1.08915286e-04],
        [2.00527671e-02, 2.37467012e-02, 2.00527671e-02, 2.16358853e-02,
         6.01250373e-04],
        [2.42744052e-02, 2.58575172e-02, 2.21635893e-02, 2.48021092e-02,
         6.29112423e-04],
        ...,
        [1.79419514e-01, 2.08443271e-01, 1.79419514e-01, 1.88390495e-01,
         6.81322105e-03],
        [1.90501311e-01, 1.95250659e-01, 1.77836415e-01, 1.78891823e-01,
         1.20883303e-03],
        [1.76781007e-01, 1.84168863e-01, 1.72559375e-01, 1.79419514e-01,
         4.16664292e-04]],

       [[4.31620231e-01, 4.63959054e-01, 4.21820563e-01, 4.47952984e-01,
         2.07224966e-03],
        [4.51219527e-01, 4.54486069e-01, 4.05487810e-01, 4.15287479e-01,
         2.27700752e-03],
        [4.21820563e-01, 4.41419858e-01, 4.21167238e-01, 4.25087105e-01,
         1.34347535e-04],
        ...,
        [3.46689883e-01, 3.64655928e-01, 3.29703847e-01, 3.36890256e-01,
        

In [ ]:
class Inline(admin.TabularInline):
  '''Tabular Inline View for '''

  model =
  min_num = 3
  max_num = 20
  extra = 1
  raw_id_fields = (,)

In [105]:
sliding_size = 1
predict_size = 15
window_size = 90
ticker_number =len(data['Ticker'].unique())
total_day = len(data['Date'].unique())
window_number =int(total_day - (window_size+predict_size)/sliding_size+1)
feature_size = len(feature)

In [109]:
feature_size

5

In [79]:
result[0,0,:].mean().item()

1.1004127264022827

In [59]:
result_cpu[0]

array([[1.17999995, 1.19000006, 1.15999997, 1.16999996, 0.80206396],
       [1.17999995, 1.25      , 1.17999995, 1.21000004, 0.81139371],
       [1.25999999, 1.28999996, 1.22000003, 1.26999998, 0.81192169],
       ...,
       [0.93000001, 0.93000001, 0.87      , 0.91000003, 0.80013801],
       [0.98000002, 0.98000002, 0.91000003, 0.91000003, 0.80002401],
       [0.94999999, 0.97000003, 0.87      , 0.93000001, 0.80012001]])

In [60]:
result[0]

tensor([[1.1800, 1.1900, 1.1600, 1.1700, 0.8021],
        [1.1800, 1.2500, 1.1800, 1.2100, 0.8114],
        [1.2600, 1.2900, 1.2200, 1.2700, 0.8119],
        ...,
        [0.9300, 0.9300, 0.8700, 0.9100, 0.8001],
        [0.9800, 0.9800, 0.9100, 0.9100, 0.8000],
        [0.9500, 0.9700, 0.8700, 0.9300, 0.8001]], device='cuda:0')

In [55]:
result_cpu =inverse_min_max(combine,min_max_list)

In [2]:
len(dfs.keys())

2136

# PUSH CSV TO HUGGINGFACE 

In [16]:
import datasets
import pandas as pd


dataset = datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y')
data= pd.read_csv('NASDAQ_3y/stock_price_clean-train.csv',index_col=0)
data.to_csv('NASDAQ_3y/stock_price_clean-train.csv',index=False)
dataset.push_to_hub('sehyun66/STOCKPRICE','NASDAQ_3y')

Generating train split: 100%|██████████| 1612680/1612680 [00:01<00:00, 1181662.92 examples/s]
